In [62]:
import requests
def fetchData():
    api_url = "https://api.uiucbus.com/api/getdeparturesbystop?stop_id=1STSTDM"
    response = requests.get(api_url)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return ""
data = fetchData()

In [67]:
print("---DATA DICTIONARY---")
dataKeys = data.keys()
for key in dataKeys:
    print(f"{key}: TYPE: {type(data[key])} , EX: {data[key]}")

time: TYPE: <class 'str'> , EX: 2025-11-24T17:24:08-06:00
new_changeset: TYPE: <class 'bool'> , EX: True
status: TYPE: <class 'dict'> , EX: {'code': 200, 'msg': 'ok'}
rqst: TYPE: <class 'dict'> , EX: {'method': 'GetDeparturesByStop', 'params': {'pt': 60, 'stop_id': '1STSTDM'}}
departures: TYPE: <class 'list'> , EX: [{'stop_id': '1STSTDM:2', 'headsign': '10E Gold', 'route': {'route_color': 'c7994a', 'route_id': 'GOLD', 'route_long_name': 'Gold', 'route_short_name': '10', 'route_text_color': '000000'}, 'trip': {'trip_id': '[@6.0.51699430@][31][1716921762883]/10__G1_MF', 'trip_headsign': 'Eastern & Perkins', 'route_id': 'GOLD', 'block_id': 'G1 MF', 'direction': 'East', 'service_id': 'G1 MF', 'shape_id': '[@6.0.51699430@]102'}, 'vehicle_id': '1603', 'origin': {'stop_id': 'E14:2'}, 'destination': {'stop_id': 'PKNSEAST:2'}, 'is_monitored': True, 'is_scheduled': True, 'is_istop': True, 'scheduled': '2025-11-24T17:26:56-06:00', 'expected': '2025-11-24T17:26:56-06:00', 'expected_mins': 3, 'loca

In [66]:
print("---DEPARTURE DICTIONARY---")
departures = data.get("departures", [])
depKeys = departures[0].keys()
for k in depKeys:
    print(f"{k}: type: {type(departures[0][k])}, ex: {departures[0][k]}")

stop_id: type: <class 'str'>, ex: 1STSTDM:2
headsign: type: <class 'str'>, ex: 10E Gold
route: type: <class 'dict'>, ex: {'route_color': 'c7994a', 'route_id': 'GOLD', 'route_long_name': 'Gold', 'route_short_name': '10', 'route_text_color': '000000'}
trip: type: <class 'dict'>, ex: {'trip_id': '[@6.0.51699430@][31][1716921762883]/10__G1_MF', 'trip_headsign': 'Eastern & Perkins', 'route_id': 'GOLD', 'block_id': 'G1 MF', 'direction': 'East', 'service_id': 'G1 MF', 'shape_id': '[@6.0.51699430@]102'}
vehicle_id: type: <class 'str'>, ex: 1603
origin: type: <class 'dict'>, ex: {'stop_id': 'E14:2'}
destination: type: <class 'dict'>, ex: {'stop_id': 'PKNSEAST:2'}
is_monitored: type: <class 'bool'>, ex: True
is_scheduled: type: <class 'bool'>, ex: True
is_istop: type: <class 'bool'>, ex: True
scheduled: type: <class 'str'>, ex: 2025-11-24T17:26:56-06:00
expected: type: <class 'str'>, ex: 2025-11-24T17:26:56-06:00
expected_mins: type: <class 'int'>, ex: 3
location: type: <class 'dict'>, ex: {'lat

In [64]:
# Expects Data dictionary from fetchData()
# Returns a list of dictionaries with headsign and expected time info
def fetchBusInfoFromData(data):
    departures = data.get("departures", [])
    out = []

    # If no busses, return empty list
    if not departures: 
        return out
    
    # For every bus get the headsign and expected time (split into multiple fields)
    for busInfo in departures:
        busDict = {}
        busDict["headsign"] = busInfo.get("headsign", "")
        expString = busInfo.get("expected", "")
        if expString == "":
            continue
        expStringHMS = expString[-14:-6]
        busDict["expectedHMS"] = expStringHMS
        busDict["expectedH"] = int(expStringHMS[:2])
        busDict["expectedM"] = int(expStringHMS[3:5])
        busDict["expectedS"] = int(expStringHMS[6:8])
        out.append(busDict)
    
    return out

# Expects Data dictionary from fetchData()
# Returns a dictionary with current time info
def getCurTime(data):
    timeStr = data.get("time", {})
    timeDict = {}
    if not timeStr:
        return timeDict
    timeStrHMS = timeStr[-14:-6]
    timeDict["timeStrHMS"] = timeStrHMS
    timeDict["hour"] = int(timeStrHMS[:2])
    timeDict["min"] = int(timeStrHMS[3:5])
    timeDict["sec"] = int(timeStrHMS[6:8])
    return timeDict

# busList = fetchBusInfoFromData(data)
# print(busList)
# print(getCurTime(data))

In [65]:
## Timing check
for i in range(10):
    d = fetchData()
    i = fetchBusInfoFromData(d)
    t = getCurTime(d)
    i, d, t = None, None, None
# Result: Averages 0.25 seconds for fetch and parse.